In [ ]:
""""
SUMMARY of the implementation
-a board( the 9x9 grid) is the state ---> and is implemented as a class.
    it will have an actual variable named board=9x9 numpy array
- sudoku.txt

-THE DOMAIN VALUES are kind of the frontier list. Implemented in dictionary key= tuple, value = list of values

-BACKTRACK ALGORITHM IS USED WITH CONSTRAINT
    ---> HUMANLY and MAKES POSSIBLE TO GUESS AND GO BACK
- 6,670,903,752,021,072,936,960 possible solved Sudoku grids  --> Sextillion 


"""

In [2]:
import numpy as np

# In our work, the board is the state and since its not atomic, it has values and attributes
class Board(): # state
    def __init__(self,filename):
        self.board=None
        self.file=filename
        self.get_board(self.file) # open the board text file given in the format

        # this keeps the list of empty cells, kind of like variables in CSP and is updated 
        self.list_of_empty_cells=None
        
        self.get_list_of_empty_cells()
        
        
        self.solvability_condition=self.is_sudoku_valid() # before any step is take, check solvability
        
        
        # the domains are kind of the frontier 
        self.domain_values={}  # this dicrionary keeps a cell with its possible values = DOMAIN !!!
        self.determine_domain_values() # this fills each empty cell with its possible value
        
        self.cost_of_movement=0
        self.clues=81-len(self.list_of_empty_cells)
        
        
    
    # is the puzzle valid ? checks to before playing the game
    def is_sudoku_valid(self):
        if (len(self.list_of_empty_cells) > 64): # this is because of minimum clues
            print("No solution")
            return False
            
        for i in self.board: 
            temp_1=[num for num in i if num]
            temp_2=list(set(i)-{0})
            if len(temp_1) != len(temp_2):
                print("No solution")
                return False
        for column in self.board.T:
            temp_1=[num for num in column if num]
            temp_2=list(set(column)-{0})
            if len(temp_1) != len(temp_2):
                print("No solution")
                return False
        return True
        
    
        
        
    # this method creates (tuple : list) where tuples are location and list is the domain
    # given a cell, what are the possibilities
    def determine_domain_values(self):     
      
      for i in range(9):
        for j in range(9):
            self.domain_values[(i,j)]=self.get_the_valid_values_for_a_cell((i,j))
 
     
    
    # list of empty cell from all over the board, it returns a list of tuples and is used at thh start
    def get_list_of_empty_cells(self):
      self.list_of_empty_cells=[]
      for i in range(9):
        for j in range(9):
          if self.board[i][j]==0:
            self.list_of_empty_cells.append((i,j))

    

    
    def get_the_valid_values_for_a_cell(self,cell):
      valid_values=[] 
      for i in range(1,10):
        if self.is_valid(i,cell):
          valid_values.append(i)

      return valid_values
    
    
    #returns a tuple location where the domains are minimum, i.e most constrained
    # this checks only those cells that are empty thus further prunning the tree

    def get_the_cell_with_the_min_domain(self):
      min_domain_index=0
      size_=len(self.list_of_empty_cells)

      for i in range(size_):
        min_domain=self.list_of_empty_cells[min_domain_index]
        new_domain= self.list_of_empty_cells[i]
        
        d1=len(self.domain_values[(min_domain[0],min_domain[1])])
        d2=len(self.domain_values[(new_domain[0],new_domain[1])])

        if d1 > d2:
          min_domain_index=i
      return self.list_of_empty_cells[min_domain_index]    
    
    
    def add_to_list_of_empty_cells(self,position):
      self.list_of_empty_cells.append(position)

    
    def get_board(self,file_name):
    
          board_file=open(file_name,'r')
          temp_board=[]

          self.board=np.zeros((9,9))
          for i in board_file:
            #print(type(i), " here")
            temp=i.strip()
            temp_board.append(temp)



          for k in range(9):
            for j in range(9):
                #print(k,j)
                if temp_board[k][j] != '0':
                    self.board[k][j]=ord( temp_board[k][j])-48

  

    def print_board(self):
      print(self.board)
    
    # a completness test
    def is_complete(self):
      return not (0 in self.board)

 

    def check_sudoku(self):
      
      if self.is_complete() is False:
        return False
      for i in range(9):
        # j, k index top left hand corner of each 3x3 tile
        j, k = (i // 3) * 3, (i % 3) * 3
        if (len(set(self.board[i,:])) != 9):
          return False
        if ( len(set(self.board[:,i])) != 9):
          return False
        if (len(set(self.board[j:j+3, k:k+3].ravel())) != 9):
           return False
      return True
 
    

    # checks if a value is valid to be put at the position= tuple
    def is_valid(self, value, position): 
          row,col = position          
          row_vals = self.board[row]
          if value in row_vals:
              return False # if we've repeated, then our guess is not valid!

              
          # checking validity of the column
   
          col_vals = [self.board[i][col] for i in range(9)]
          if value in col_vals:
              return False

          # checking validity of the block
          row_start = (row // 3) * 3 # 10 // 3 = 3, 5 // 3 = 1, 1 // 3 = 0
          col_start = (col // 3) * 3

          for r in range(row_start, row_start + 3):
              for c in range(col_start, col_start + 3):
                  if self.board[r][c] == value:
                      return False
          return True
   


    def assign_val_to_a_cell(self,val,row,col): # this is a move the agent makes
      self.board[row][col]=val
      self.list_of_empty_cells.remove((row,col))
      
      # upon every assignmet, make cutting domains, this is forward checking
      self.determine_domain_values()
      self.cost_of_movement+=1
        
        

    

    def undo_assignment(self,val,row,col):    
      self.board[row][col]=0
      self.cost_of_movement+=1
      self.list_of_empty_cells.append((row,col)) # add to empty cells
      self.determine_domain_values()
      
    # there are two kinds of assignments: one is legal where it is sure to satisfy the 
    # constraints, ... the first steps of the algorithm should check those steps that already satisfy


    
    
    #Sudoku solver, recursive function
    def solve_sudoku(self):
        
        if not self.solvability_condition:
            print("Unsolvable sudoku")
            return False
            
        
        
        if self.check_sudoku():
            return True

        else:
            next_cell= self.get_the_cell_with_the_min_domain() #Constraint propagation is implemented here

            candidates=self.domain_values[(next_cell[0],next_cell[1])]

            for i in candidates:
                self.assign_val_to_a_cell(int(i),next_cell[0],next_cell[1])
                self.solve_sudoku()
                if self.check_sudoku():
                    return True
                self.undo_assignment(i,next_cell[0],next_cell[1])
               

        
        return False
    



In [3]:
# sudoku_1.txt
# sudoku2.txt

main_board=Board('sudoku_1.txt')
main_board.print_board()
print('\n')
print("is it solved ? :",main_board.check_sudoku())
print("Moves made : ",main_board.cost_of_movement)
print("clues given : ", main_board.clues)
print("Remaining values : ", 81-main_board.clues)

[[2. 0. 0. 0. 8. 0. 3. 0. 0.]
 [0. 6. 0. 0. 7. 0. 0. 8. 4.]
 [0. 3. 0. 5. 0. 0. 2. 0. 9.]
 [0. 0. 0. 1. 0. 5. 4. 0. 8.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [4. 0. 2. 7. 0. 6. 0. 0. 0.]
 [3. 0. 1. 0. 0. 7. 0. 4. 0.]
 [7. 2. 0. 0. 4. 0. 0. 6. 0.]
 [0. 0. 4. 0. 1. 0. 0. 0. 3.]]


is it solved ? : False
Moves made :  0
clues given :  30
Remaining values :  51


In [4]:
main_board.solve_sudoku()

main_board.print_board()
print("is it solved? : ",main_board.check_sudoku())
print("Moves made : ",main_board.cost_of_movement)
print(main_board.clues)

[[2. 4. 5. 9. 8. 1. 3. 7. 6.]
 [1. 6. 9. 2. 7. 3. 5. 8. 4.]
 [8. 3. 7. 5. 6. 4. 2. 1. 9.]
 [9. 7. 6. 1. 2. 5. 4. 3. 8.]
 [5. 1. 3. 4. 9. 8. 6. 2. 7.]
 [4. 8. 2. 7. 3. 6. 9. 5. 1.]
 [3. 9. 1. 6. 5. 7. 8. 4. 2.]
 [7. 2. 8. 3. 4. 9. 1. 6. 5.]
 [6. 5. 4. 8. 1. 2. 7. 9. 3.]]
is it solved? :  True
Moves made :  61
30
